In [ ]:
# install all libraries
!pip install mmdet==2.3.0 requests
!pip install torch==1.5.1+cpu torchvision==0.6.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
!pip install mmcv-full==1.0.5 -f https://download.openmmlab.com/mmcv/dist/cpu/torch1.5.0/index.html
!pip install pdf2image 
!apt-get install poppler-utils
!pip install img2pdf
!pip install ocrmypdf
!pip install camelot-py
!pip install pytesseract
!apt install ghostscript
!pip install ghostscript
!apt install tesseract-ocr
!apt install libtesseract-dev

In [ ]:
# download and load model

conf_url = 'https://raw.githubusercontent.com/iiLaurens/CascadeTabNet/mmdet2x/Config/cascade_mask_rcnn_hrnetv2p_w32_20e.py'  #Configuration file 
with open('cascade_mask_rcnn_hrnetv2p_w32_20e.py', 'wb') as f: 
  f.write(requests.get(conf_url).content)

#model_name = 'ICDAR.19.Track.B2.Modern.table.structure.recognition.v2.pth'     #317Mb                           #Table and cell detection model(Table structure model)
model_name = 'ICDAR.19.Track.A.Modern.table.detection.v2.pth'                   #317Mb                           #Only table detection

checkpoint_url = 'https://github.com/iiLaurens/CascadeTabNet/releases/download/v1.0.0/' + model_name

with open(model_name, 'wb') as f:
  f.write(requests.get(checkpoint_url).content)

In [ ]:
# load model
config_file = 'cascade_mask_rcnn_hrnetv2p_w32_20e.py'
checkpoint_file = model_name
model = init_detector(config_file, checkpoint_file, device='cpu')

In [ ]:
# load libraries
from os.path import basename
import os
import json
from werkzeug.utils import secure_filename
from flask_cors import CORS
from flask import Flask, request, jsonify, Response
from dotenv import load_dotenv
from pdf2image import convert_from_path
import pytesseract
import camelot
import ocrmypdf
import ghostscript
import PIL
import img2pdf
import numpy as np
import requests
import mmcv
import cv2
import glob
from mmdet.apis import init_detector, inference_detector, show_result_pyplot

In [ ]:
def extract_table_img(image, table):
    x1, y1, x2, y2, score = table
    cropped_image = image[int(y1)-5:int(y2)+5, int(x1)-7:int(x2)+7]                     
    return cropped_image

def extract_tables(input_file_name):
    # try:
    #     files = glob.glob('table_image/*');
    # except:
    #     pass
    
    # try:
    #     for f in files:
    #         os.remove(f)
    # except:
    #     pass

        
    pages = convert_from_path(input_file_name, dpi = 300, fmt = "ppm")
    list_of_json_file=[]
    count_=1
    page_num = 1                                                                                                                     
    for page in pages:                                                                                                              #Looping over pages of pdf
        image = np.array(page)                                                                                                      
        image = image[:, :, ::-1]                                                                                                   
        result = inference_detector(model, image)                                                                                   
        if (len(result[0][0]) > 0):
            for table_no, table in enumerate(result[0][0]):                                                                         #Cropping every table in the page
                extracted_table = extract_table_img(image, table)                                                                   #Extracting cropped image 

                image_name = str(page_num) + "table_image_" + str(table_no) + "_" + basename(input_file_name)[:-4] + ".jpg"
                print(image_name)
                cv2.imwrite('table_image/'+image_name, extracted_table)
                pdf_bytes = img2pdf.convert('table_image/'+image_name)                                                                           #Converting extracted table image to pdf
                
                pdf_name = str(page_num) + "table_pdf_" + str(table_no) + "_" + basename(input_file_name)[:-4] +  ".pdf"
                file = open('pdf_table/'+pdf_name, "wb")
                file.write(pdf_bytes)

                ocrmypdf.ocr('pdf_table/'+pdf_name, 'pdf_table/'+pdf_name)                                                                                     #Performing OCR on pdf
                try:
                    tables = camelot.read_pdf('pdf_table/'+pdf_name, flavor='stream',  row_tol = 20)                                             
                    df = tables[0].df         #Converting table into a dataframe
                    df.to_excel('excel_file/'+str(count_).xlsx)
                    result = json.loads(df.to_json(orient="index"))
                    image_path = os.path.abspath('table_image/'+image_name)
                    # append table data
                    list_of_json_file.append({"table_"+str(count_):str(result),"image_url_"+str(count_):image_path})
                    count_ = count_+1                                                                                       

                except:
                    print("Table not detected by Camelot")  
        page_num +=1

    # clean folders
    # try:
    #     files = glob.glob('pdf_table/*');
    # except:
    #     pass
    
    # try:
    #     for f in files:
    #         os.remove(f)
    # except:
    #     pass
    

    return list_of_json_file

In [ ]:
if __name__ == "__main__": 
    extract_tables("pdf_file.pdf")